In [1]:
import sys 
sys.path.append('E:/xinlib')
import xcv
from xcv import XCV

from mxnet import gluon, nd, autograd, image
from mxnet.gluon import nn
from gluoncv.model_zoo import get_model
import mxnet as mx
import numpy as np

batch_size = 64
    
resize = 32
prob = 0.5
loader = XCV(batch_size, prob, resize)
annotations = loader.unzip_lables()
annotations

['E:\\Data\\Badidu\\datasets\\test.txt',
 'E:\\Data\\Badidu\\datasets\\train.txt']

In [2]:
_, n_class = loader.train_(annotations[1])
n_class

100

In [3]:
class X(nn.HybridBlock):
    '''
    此网络有两个输出
    '''
    def __init__(self, n_class, **kwargs):
        super().__init__(**kwargs)
        self.n_class = n_class
        
        with self.name_scope():
            self.features = get_model(
                'cifar_wideresnet40_8', classes=10, pretrained=False).features[:6]
            self.features.add(
                nn.BatchNorm(axis=1),
                nn.Activation(activation='relu'),
                nn.Conv2D(channels=256, kernel_size=1, padding=0),

                nn.BatchNorm(axis=1),
                nn.Conv2D(channels=self.n_class, kernel_size=1, padding=0))
            self.out = nn.HybridSequential()
            self.out.add(
                nn.GlobalAvgPool2D(),
                nn.Flatten())
            self.outimg = nn.HybridSequential(prefix='out_image')
            self.outimg.add(
                nn.BatchNorm(axis=1),
                nn.Conv2D(channels=50, kernel_size=1, padding=0),
                nn.Activation(activation='relu'),

                nn.BatchNorm(axis=1),
                nn.Conv2D(channels=3, kernel_size=1, padding=0),
            )

    def hybrid_forward(self, F, x):
        x = self.features(x)
        out = self.out(x)
        imgout = self.outimg(x)
        return out, imgout
    
net = X(n_class)

In [4]:
loader.transform_train = loader.transform_test

In [5]:
loader.train(annotations[1], net, 1)

INFO:mxboard.event_file_writer:successfully opened events file: D:/logs/cross_entropy/x0\events.out.tfevents.1525955458.Xinet
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:successfully opened events file: D:/logs\events.out.tfevents.1525955459.Xinet
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 13 events to disk


[Epoch 0] Training: accuracy=0.0418502: 46.78872323036194 s

INFO:mxboard.event_file_writer:wrote 1 event to disk


INFO:mxboard.event_file_writer:successfully opened events file: D:/logs/accuracy_curves/x0_train_acc\events.out.tfevents.1525955495.Xinet
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk


[Epoch 0] Validation: accuracy=0.00880411


INFO:mxboard.event_file_writer:successfully opened events file: D:/logs/accuracy_curves/x0_valid_acc\events.out.tfevents.1525955517.Xinet
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 8 events to disk


In [6]:
import json

def get_result():
    with open('D:/logs/%s_scalar_dict.json'%net.name, encoding='utf-8') as f:
        return json.load(f)

In [7]:
def get_params(net, k):
    '''
    k::模型参数的序号
    '''
    fname = loader.logdir + '/models/%s-%04d.params' % (net.name,k)
    params = net.load_params(fname, ctx=loader.ctx) 
    return params

In [8]:
get_params(net, 0)

AssertionError: Parameter 'cifarwideresnet0_batchnorm0_gamma' is missing in file 'D:/logs/models/x0-0000.params', which contains parameters: 'x0_x0_cifarwideresnet0_batchnorm0_gamma', 'x0_x0_cifarwideresnet0_batchnorm0_beta', 'x0_x0_cifarwideresnet0_batchnorm0_running_mean', ..., 'x0_x0_batchnorm3_running_mean', 'x0_x0_batchnorm3_running_var', 'x0_x0_conv3_weight', 'x0_x0_conv3_bias'. Please make sure source and target networks have the same prefix.

In [44]:
a = 0.7
b  = a * .25**3
b

0.0109375

In [14]:
a * .74

0.22200000000000003